Mobile에서 서버로 request를 보내고, shutdown메세지를 보내고 (send는 끝났고 이제는 듣기만 하겠다), sever의 메세지를 기다렸다 받으면 됨.  
서버의 메세지를 성공적으로 받으면 close로 연결 종료. 


In [1]:
import socket
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
LOCALHOST = '127.0.0.1'
port = 9990

SEPARATOR = "<sep>"

server_socket.bind((LOCALHOST,port))
server_socket.listen(5)

print("Server started...")

client_sockets,addr=server_socket.accept()
while True:
    msg_received = client_sockets.recv(1024)
    msg_received = msg_received.decode()
    print("Client:", msg_received)
    if msg_received == "ByeBye":
        break
    
    ########################
    received = client_sockets.recv(1024) #1024 Byte를 채우려고 뒤에 오는 binary 파일까지 이 버퍼에 받아버림. 
    print(received)
    received = received.decode()
    filname, filesize = received.split(SEPARATOR)
    filename = os.path.basename(filename)
    filesize = int(filesize)
    with open(filename+"_received", "wb") as f:
        bytes_read = client_socket.recv(BUFFER_SIZE)
        
        if not bytes_read:
            break
        
        f.write(bytes_read)
    ###########
    msg_send = input("Me:")
    client_sockets.send(msg_send.encode("ascii"))

client_sockets.close()

Server started...
Client: 1
b'/home/hoseung/Work/fhenrf/pose/rav_example.pickle<sep>2553\x80\x04\x95\xee\t\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x01M,\x01\x86\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B`\t\x00\x00\x00\x00\x00\x00\x00\xf0p@\x00\x00\x00\x00\x00\xe0o@\x00\x00\x00\x00\x000p@\x00\x00\x00\x00\x00@n@\x00\x00\x00\x00\x00\xa0o@\x00\x00\x00\x00\x00\x80k@\x00\x00\x00\x00\x00\xa0n@\x00\x00\x00\x00\x00\xc0j@\x00\x00\x00\x00\x00@n@\x00\x00\x00\x00\x00\xc0m@\x00\x00\x00\x00\x00@o@\x00\x00\x00\x00\x000p@\x00\x00\x00\x00\x000p@\x00\x00\x00\x00\x00\xc0i@\x00\x00\x00\x00\x00\x90o@\x00\x00\x00\x00\x00\xb0j@\x00\x00\x00\x00\x00\x00o@\x00\x00\x00\x00\x00\x10p@\x00\x00\x00\x00\x00 p@\x00\x00\x00\x00\x00Ps@\x00\x00\x00\x00\x00\x10q@\x00\x00\x00\x00\x00pq@\x00\x0

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 58: invalid start byte

In [ ]:
import tqdm
import os
import socket

HOST = socket.gethostname()
#HOST = '10.100.82.54'
PORT = 9999

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR,1)

server_socket.bind((HOST,PORT))
server_socket.listen(5)

client_socket, addr = server_socket.accept()
print('Conneted by',addr)


# A socket is "discarded" on returning 0 
# If not, the socket will wait for the next message forever.
# This means that the end of a conversation need to be marked somehow. 
# 1. Use fixed-lengh message
# 2. Use delimiter 
# 3. the length of message told beforehand
# 4. or just shutdown the connection.

SEPARATOR = "<sep>"
BUFFER_SIZE=4096


received = client_socket.recv(BUFFER_SIZE).decode()
filename, filesize = received.split(SEPARATOR)

received2 = client_socket.recv(BUFFER_SIZE).decode()
filename2, filesize2 = received.split(SEPARATOR)
print(filename2, filesize2)

# remove absolute path if there is
filename = os.path.basename(filename)
# convert to integer
filesize = int(filesize)

progress = tqdm.tqdm(range(filesize), f"Receiving {filename}", unit="B", unit_scale=True, unit_divisor=1024)
with open(filename+"_received", "wb") as f:
    while True:
        # read 1024 bytes from the socket (receive)
        bytes_read = client_socket.recv(BUFFER_SIZE)
        if not bytes_read:    
            # nothing is received
            # file transmitting is done
            break
        # write to the file the bytes we just received
        f.write(bytes_read)
        # update the progress bar
        progress.update(len(bytes_read))

# close the client socket
client_socket.close()

Conneted by ('127.0.0.1', 38996)
/home/hoseung/Work/fhenrf/pose/rav_example.pickle 2553


Receiving rav_example.pickle: 100%|█████████████████████████████████████████| 2.49k/2.49k [00:17<00:00, 144B/s]


In [2]:
pwd

'/home/hoseung/Work/fhe-ai-sw-etri/fase/notebooks'

## Client 메세지 받기 

1. 정해진 길이를 사용할지, delimiter를 쓸지, buffer 길이 변수를 메세지 앞에 달지 결정. 
2. Client가 메세지를 다 보냈는지 확인 (EOF 메세지가 올 것)
3. 


In [4]:
socket.gethostbyname(HOST)

'10.100.82.54'

## Interpret received file 

Make sure to check Endianess

In [126]:
fn_out = "./enc.dat"
f = open(fn_out, 'rb')

bstr = f.read(1000)

In [48]:
bstr=b'8304221185600'

In [49]:
bstr

b'8304221185600'

In [83]:
import struct

In [108]:
import sys

In [110]:
my_bytearray

bytearray(b'\xd6\xab\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [111]:
s_str: str = "\x00\x01\x00\xc0\x01\x00\x00\x00\x04"
print_code(s_str)

ASCII:   À   
HEX: 000100c3800100000004


In [132]:
secret = 830422 #1185600

print_code(secret)

ASCII: 830422


In [133]:
some_bytes = secret.to_bytes(32, sys.byteorder)
my_bytearray = bytearray(some_bytes)

print_code(my_bytearray)

ASCII: bytearray(b'\xd6\xab\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
HEX: d6ab0c0000000000000000000000000000000000000000000000000000000000


In [135]:
fn_out = "./enc.dat"
f = open(fn_out, 'rb')
bstr = f.read(1000) # first 1000 Bytes

print_code(bstr)

ASCII: b'^\xa1\x10\x03\x07\x00\x00\x00i\x008\x00\x00\x00\x00\x007\x87gD;\x977.\xf9\xde6i\xa6\xe7K\x0e\xf1\xdc@\xab\xbaj\xaf\x1f\xfd\t\x86\xa2I,J\xda\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb0A^\xa1\x10\x03\x07\x00\x00\x00\x18\x008\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x99\xd6\x03\x04\x18\x00\x00\x00\x0b\\\x12\xae\x1b\x00\x00\x00\x80\x8c\x1d5\x07\x00\x00\x00\\\xe4GL\x16\x00\x00\x00\xf7\x88\x8e\x7f\x0b\x00\x00\x00\xfa\x18\x87\xfe\x06\x00\x00\x00\x1e\xfd_-\x07\x00\x00\x00\x84"[\xd3\x05\x00\x00\x003\x98\x85\x91\x1a\x00\x00\x00W\t\x0c\t\x1d\x00\x00\x00]d\xcb\xd4\x0e\x00\x00\x00U\xd7:\x83\x0f\x00\x00\x00KA\xc7\xa8\x10\x00\x00\x00\x1c\xb0\x9f%\x07\x00\x00\x00%l\x91?\x1c\x00\x00\x00\x17\x8f")\x12\x00\x00\x00\x9d\x8e\x99\x19\x02\x00\x00\x00W\xe3\xd8\x82\x08\x00\x00\x00\x03\xfb*\x03\x13\x00\x00\x00\xb0\x9a\xcd\xba\x00\x00\x00\x00\x94\xe8\xc0\xda\x1a\x00\x00\x00JY\x8c\x00\x13\x00\x00\x00\xe0P\xb7\xa1\x

In [131]:
def print_code(arr):
    #print(arr)
    print("ASCII:", arr)
    try:
        if isinstance(arr, bytearray):
            #print("ASCII:", arr.decode("ascii", errors='ignore'))
            print("HEX:", arr.hex())
        else:
            if not isinstance(arr, bytes):
                arr = bytes(arr, encoding='utf')
            #print("ASCII:", arr.decode("ascii", errors='ignore'))
            print("HEX:", arr.hex())
    except:
        pass

In [115]:
print_code(secret)

ASCII: 830422


TypeError: encoding without a string argument

In [116]:
print_code(bstr)

ASCII: b'\x00\x00\x00\x00\x00\x99\xd6\x03\x04\x18\x00\x00\x00\x0b\\\x12\xae\x1b\x00\x00\x00\x80\x8c\x1d5\x07\x00\x00\x00\\\xe4GL\x16\x00\x00\x00\xf7\x88\x8e\x7f\x0b\x00\x00\x00\xfa\x18\x87\xfe\x06\x00\x00\x00\x1e\xfd_-\x07\x00\x00\x00\x84"[\xd3\x05\x00\x00\x003\x98\x85\x91\x1a\x00\x00\x00W\t\x0c\t\x1d\x00\x00\x00]d\xcb\xd4\x0e\x00\x00\x00U\xd7:\x83\x0f\x00\x00'
HEX: 000000000099d60304180000000b5c12ae1b000000808c1d35070000005ce4474c16000000f7888e7f0b000000fa1887fe060000001efd5f2d0700000084225bd305000000339885911a00000057090c091d0000005d64cbd40e00000055d73a830f0000


In [46]:
print(bstr)

8304221185600


In [136]:
import os

In [137]:
os.path.join("/home/hoseung/Work/data/BBS", "/g1/a/001/")

'/g1/a/001/'

In [123]:
from fase import seal

In [124]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)
parms.load('./prams.sav')

context = seal.SEALContext(parms)

In [125]:
ctx = seal.Ciphertext()
ctx.load(context, fn_out)

In [ ]:
ctc = 

In [35]:
ctx.__

'<fase.seal.Ciphertext object at 0x7f6f7b8ab9b0>'

In [ ]:
decryptor = seal.Decryptor()